In [ ]:
from sklearn import datasets, svm
from sklearn.model_selection import KFold, GridSearchCV

iris = datasets.load_iris()

# clf = svm.SVC()
# clf.fit(X, y)
estimators = []
test_scores = []
train_scores = []

kf_outer = KFold(n_splits=2)
for i, (train_index, test_index) in enumerate(kf.split(iris.data)):
    parameters = {'kernel':('linear', 'rbf'), 'C':[1, 10]}
    svc = svm.SVC()
    clf = GridSearchCV(svc, parameters, cv=5)
    clf.fit(iris.data[train_index], iris.target[train_index])
    estimators.append(clf.best_estimator_)
    test_scores.append(clf.score(iris.data[test_index], iris.target[test_index]))

max_score = max(test_scores)
max_score_index = test_scores.index(max_score)

fitted_estim = estimators[max_score_index]

fitted_estim.score(iris.data, iris.target)